In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import frame
import YCoCg

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_frame(frame, prefix=''):
    #frame = normalize(frame)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(frame)

In [ ]:
x = frame.read("../sequences/stockholm/", 0)

In [ ]:
x.max()

In [ ]:
x.min()

In [ ]:
show_frame(x.astype(np.uint8))

In [ ]:
y = YCoCg.from_RGB(x)

In [ ]:
show_frame(y.astype(np.uint8))

In [ ]:
z = YCoCg.to_RGB(y)

In [ ]:
z.max()

In [ ]:
z.min()

In [ ]:
show_frame(z.astype(np.uint8))

In [ ]:
print(np.array_equal(x, z))